In [139]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv(r"data/session_prepared_binCoder.csv", index_col=0);

In [3]:
df.head()

,is_target_action,screen_area_scl,utm_source_0,utm_source_1,utm_source_2,utm_source_3,utm_source_4,utm_source_5,utm_source_6,utm_source_7,...,geo_city_1,geo_city_2,geo_city_3,geo_city_4,geo_city_5,geo_city_6,geo_city_7,geo_city_8,geo_city_9,geo_city_10
0,0,-0.343096,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,0,0.042773,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,0,-0.343096,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,1
5,0,-0.091912,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,1,0,0
6,0,1.654497,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [15]:
x = df.drop(columns=['is_target_action'])
y = df.is_target_action

In [38]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=23)

In [39]:
model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

In [57]:
threshold = 0.085
y_pred_bool = np.array([1 if x > threshold else 0 for x in y_pred])
roc_auc_score(y_test, y_pred_bool)

0.6678718034128949

In [15]:
confusion_matrix(y_test, y_pred_bool)

array([[48722, 12176],
       [ 2074,  2069]], dtype=int64)

In [17]:
def roc_auc(estimator, x, y):
    y_pred = estimator.predict(x)
    return roc_auc_score(y, y_pred)

In [142]:
class CustomClassifier(LinearRegression):
    def __init__(self, threshold=0.08, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
    
    def predict(self, X):
        pred = super().predict(X)
        return np.array([1 if x > self.threshold else 0 for x in pred])

In [143]:
for thresh in np.arange(0, 0.2, 0.01):
    est = CustomClassifier(threshold=thresh)
    scores = cross_val_score(est, x, y, scoring=roc_auc, cv=5)
    print(f"thresh= {thresh}  min= {np.min(scores)}  mean= {np.mean(scores)}  max= {np.max(scores)}")

thresh= 0.0  min= 0.518027622842033  mean= 0.5344371542636552  max= 0.5471423279941461
thresh= 0.01  min= 0.5257739985473509  mean= 0.5475624448769569  max= 0.5634595040069388
thresh= 0.02  min= 0.5351705694874972  mean= 0.5639985786160103  max= 0.5823280460398588
thresh= 0.03  min= 0.5463291011347335  mean= 0.5844129697277651  max= 0.6048467086505251
thresh= 0.04  min= 0.5617298088333011  mean= 0.6061445116372296  max= 0.6293218038520818
thresh= 0.05  min= 0.5745370944690541  mean= 0.6271862915017633  max= 0.6519747658491628
thresh= 0.06  min= 0.5921937818402838  mean= 0.6453444664357862  max= 0.668217759511052
thresh= 0.07  min= 0.6133796045461741  mean= 0.657930613091249  max= 0.6744798123224599
thresh= 0.08  min= 0.63249138903017  mean= 0.6640548314367914  max= 0.6785167222916327
thresh= 0.09  min= 0.6476936869193777  mean= 0.660332774351032  max= 0.6691188022538418
thresh= 0.1  min= 0.6252539663726069  mean= 0.642227543382411  max= 0.65714148038929
thresh= 0.11  min= 0.59113775621

KeyboardInterrupt: 

## План

Обучить модель на всех данных

Упаковать модель в сервис

## Параметры

Регрессор:LinearRegression
Порог = 0.08